In [25]:
# Initial population.   CHECK
# Fitness function.   CHECK
# Selection.    CHECK
# Crossover.    CHECK
# Mutation.   CHECK

In [26]:
from random import sample, randrange, randint, uniform


In [27]:
def parentSelected(populationBest, populationWorst, fitnessBest, fitnessWorst):
  temp_best = fitnessBest.copy()
  temp_worst = fitnessWorst.copy()

  threshold_best = sorted(temp_best)[round(len(populationBest)*70/100)]
  threshold_worst = sorted(temp_worst,reverse=True)[round(len(populationWorst)*70/100)]

  worstPopulation = populationWorst.copy()
  bestPopulation = populationBest.copy()

  for i in range(len(populationBest)):
    if(fitnessBest[i] > threshold_best):
      bestPopulation[i] = bestPopulation[randrange(len(populationBest))].copy()

  for i in range(len(populationWorst)):
    if(fitnessWorst[i] < threshold_worst):
      worstPopulation[i] = worstPopulation[randrange(len(populationWorst))].copy()

  return worstPopulation, bestPopulation

In [28]:
class Genetic:
  def __init__(self, Cities, populationSize, distances):
    self.population_best = []
    self.population_worst = []
    self.populationSize = populationSize
    self.fitness_best = [0 for i in range(populationSize)]
    self.fitness_worst = [0 for i in range(populationSize)]
    self.distances = distances
    self.Cities = Cities
    self.mutation_rate = 0.05

  def calculateFitness(self,gnome):
    f = 0
    turnAround_dis = self.distances[gnome[len(gnome)-1]][gnome[0]]
    for i in range(len(gnome)-1):
      distance = self.distances[gnome[i]][gnome[i+1]]
      f += distance 
    return f + turnAround_dis

  def reProduce(self, x, y):
    new_xy = []
    c = randrange(1,len(x))

    new_xy = x[:c]
    for i in range(len(y)):
      y_city = y[i]
      if y_city not in new_xy:
        new_xy.append(y_city)
    return new_xy

  def Mutate(self,child, mutation_rate):
    for i in range(len(child)):
      # check for a mutation
      if uniform(0, 1) < mutation_rate:

      # swap position
        a = randint(0, len(child)-1)
        b = randint(0, len(child)-1)
        child[a], child[b] = child[b], child[a]
    return child

  def GeneticAlgorithm(self):
    # generate population
    self.population_best = [sample(list(range(len(self.Cities))), len(self.Cities)) for i in range(populationSize)]
    self.population_worst = [sample(list(range(len(self.Cities))), len(self.Cities)) for i in range(populationSize)]

    # keep track of best solution
    bestPath, bestFit = self.population_best[0], sum([self.distances[self.population_best[0][j]][self.population_best[0][j+1]] for j in range(len(self.population_best[0])-1)])
    worstPath, worstFit = self.population_worst[0], sum([self.distances[self.population_worst[0][j]][self.population_worst[0][j+1]] for j in range(len(self.population_worst[0])-1)])

    # enumerate 200 generations
    for gen in range(200):

      # calculate fitness
      for i in range(self.populationSize):
        gnome = self.population_best[i]
        self.fitness_best[i] = self.calculateFitness(gnome)

      for i in range(self.populationSize):
        gnome = self.population_worst[i]
        self.fitness_worst[i] = self.calculateFitness(gnome)

      # check for new best solution
      for i in range(self.populationSize):
        if(self.fitness_best[i] < bestFit):
          bestPath, bestFit = self.population_best[i], self.fitness_best[i]

      # check for new worst solution
      for i in range(self.populationSize):
        if(self.fitness_worst[i] > worstFit):
          worstPath, worstFit = self.population_worst[i], self.fitness_worst[i]


      # # select parents
      worstSelected, bestSelected = parentSelected(self.population_best, self.population_worst, self.fitness_best, self.fitness_worst)

      # create the next generation
      nextBestPopulation = []
      nextWorstPopulation = []

      for i in range(self.populationSize):
        x_best,y_best = bestSelected[randrange(self.populationSize)], bestSelected[randrange(self.populationSize)]
        x_worst, y_worst = worstSelected[randrange(self.populationSize)], worstSelected[randrange(self.populationSize)]
        

        # Reproduce create new child
        child_best = self.reProduce(x_best, y_best) 
        child_worst = self.reProduce(x_worst, y_worst) 

        # Mutation
        mutated_child_best = self.Mutate(child_best, self.mutation_rate)
        mutated_child_worst = self.Mutate(child_worst, self.mutation_rate)

        # Store the next generation
        nextBestPopulation.append(mutated_child_best)
        nextWorstPopulation.append(mutated_child_worst)

      # Replace population
      self.population_best = nextBestPopulation
      self.population_worst = nextWorstPopulation

    return bestPath, bestFit, worstPath, worstFit

if __name__ == "__main__":
  distances= [[0,366,160,242,161,176,77,151,226,244],
              [366,0,430,289,703,190,399,520,342,172],
              [160,430,0,120,351,180,169,283,438,89],
              [242,289,120,0,623,250,192,331,720,120],
              [161,703,351,623,0,258,186,86,112,352],
              [176,190,180,250,258,0,169,262,176,175],
              [77,399,169,192,186,169,0,69,236,160],
              [151,520,283,331,86,262,69,0,193,414],
              [226,342,438,720,112,176,236,193,0,427],
              [244,172,89,120,352,175,160,414,427,0]]
  Cities = ['Bucharest', 'Arad', 'Craiova', 'Dobreta', 'Eforie', 'Fagaras', 'Giurgiu', 'Hirsova', 'Iasi', 'Lugoj']
  populationSize = 1000

In [29]:
  genetic = Genetic(Cities, populationSize, distances)

In [31]:
  bestPath, bestFit, worstPath, worstFit = genetic.GeneticAlgorithm()
  print('Shortest Path = ', [Cities[city] for city in bestPath])
  print('BestFit = ',bestFit)
  print('Longest Path = ', [Cities[city] for city in worstPath])
  print('WorstFit = ',worstFit)

Shortest Path =  ['Fagaras', 'Iasi', 'Eforie', 'Hirsova', 'Giurgiu', 'Bucharest', 'Craiova', 'Dobreta', 'Lugoj', 'Arad']
BestFit =  1282
Longest Path =  ['Hirsova', 'Lugoj', 'Bucharest', 'Fagaras', 'Giurgiu', 'Craiova', 'Iasi', 'Dobreta', 'Eforie', 'Arad']
WorstFit =  4176
